In [60]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import shutil
import difflib
from pulp import *

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'01.01.2022'

In [75]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xls", index=False)
os.remove("DK NBA Projections.csv")

C:\Users\vchang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.


In [61]:
downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data'
daily_file = "ETR_Daily_" + str(today) + ".xls"

os.chdir(data_loc)

df = pd.read_excel(daily_file)

In [57]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [76]:
## Merge

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\ETR_Daily_' + str(today) + ".xls")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position']], 
              left_on='Player', 
              right_on='Name')

In [77]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position
0,Nikola Jokic,DEN,HOU,35.5,C,"12,600",67.0,10.6,76.3,33.1,MAIN,Nikola Jokic,C/UTIL
1,Will Barton,DEN,HOU,36.0,SG,"6,200",37.8,7.0,45.4,23.5,MAIN,Will Barton,SG/G/UTIL
2,Facundo Campazzo,DEN,HOU,32.0,PG/SG,"4,200",27.1,4.3,35.1,44.6,MAIN,Facundo Campazzo,PG/SG/G/UTIL
3,Kevon Looney,GSW,UTA,26.0,C,"3,800",22.4,1.2,29.4,2.4,MAIN,Kevon Looney,C/UTIL
4,Josh Christopher,HOU,DEN,26.0,SG/SF,"3,700",21.7,0.9,30.0,7.0,MAIN,Josh Christopher,SG/SF/F/G/UTIL


In [78]:
#df.columns = df.columns.str.replace('_',' ')
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)
df['DK Salary'] = pd.to_numeric(df['DK Salary'])
df['Roster Position'] = df['Roster Position'].str.split('/')
explode = df.explode('Roster Position')
explode.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position
0,Nikola Jokic,DEN,HOU,35.5,C,12600.0,67.0,10.6,76.3,33.1,MAIN,Nikola Jokic,C
0,Nikola Jokic,DEN,HOU,35.5,C,12600.0,67.0,10.6,76.3,33.1,MAIN,Nikola Jokic,UTIL
1,Will Barton,DEN,HOU,36.0,SG,6200.0,37.8,7.0,45.4,23.5,MAIN,Will Barton,SG
1,Will Barton,DEN,HOU,36.0,SG,6200.0,37.8,7.0,45.4,23.5,MAIN,Will Barton,G
1,Will Barton,DEN,HOU,36.0,SG,6200.0,37.8,7.0,45.4,23.5,MAIN,Will Barton,UTIL


# Create the Constraint Problem

Goal: Maximize DK Salary

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [79]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [80]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [81]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 8

In [87]:
# position constraints
# TODO fix this, currently won't work
# as it makes the problem infeasible
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    
prob += get_position_sum(player_vars, df, 'PG') <= 3
prob += get_position_sum(player_vars, df, 'SG') <= 3
#prob += get_position_sum(player_vars, df, 'G') == 1
prob += get_position_sum(player_vars, df, 'SF') <= 3
prob += get_position_sum(player_vars, df, 'PF') <= 3
#prob += get_position_sum(player_vars, df, 'F') == 1
prob += get_position_sum(player_vars, df, 'C') <= 2
#prob += get_position_sum(player_vars, df, 'UTIL') == 1



In [88]:
# total salary constraint

prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [89]:
# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1668: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [90]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [91]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['C', 'UTIL'] Nikola Jokic DEN 12600.0 67.0
['SG', 'G', 'UTIL'] Will Barton DEN 6200.0 37.8
['PG', 'SG', 'G', 'UTIL'] Facundo Campazzo DEN 4200.0 27.1
['PF', 'F', 'UTIL'] Royce O'Neale UTA 3500.0 20.9
['SF', 'F', 'UTIL'] Davon Reed DEN 3100.0 18.8
['C', 'UTIL'] Christian Wood HOU 8800.0 40.2
['SF', 'PF', 'F', 'UTIL'] Rudy Gay UTA 3600.0 20.0
['PG', 'G', 'UTIL'] Kevin Porter Jr. HOU 8000.0 34.4


(50000.0, 266.2)